<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
import sys
sys.path.append('..')

In [2]:
from taiko.profile import Profile
from taiko.primitive import Primitive

In [3]:
pf = Profile('howeverover')

In [4]:
pf.profile_primitive_df

,L_A_AI,L_G_AI,L_AX_AI,L_AY_AI,L_AZ_AI,L_GX_AI,L_GY_AI,L_GZ_AI,L_A_SMA,L_G_SMA,...,R_GX_ZCR,R_GY_ZCR,R_GZ_ZCR,R_AXY_CORR,R_AYZ_CORR,R_AZX_CORR,R_GXY_CORR,R_GYZ_CORR,R_GZX_CORR,hit_type
0,0.394580,5.912502,0.067952,-0.139292,0.262124,-1.699218,0.801261,3.200713,0.590474,9.280589,...,0.000000,0.000000,0.05,-0.162001,0.233607,0.129188,0.393108,0.107346,-0.654075,1
1,0.521990,12.433249,0.026320,-0.164263,0.477428,-8.105777,8.537009,1.933467,0.733978,18.970224,...,0.000000,0.000000,0.00,0.518639,0.651491,0.863587,0.196854,0.090688,-0.841464,1
2,0.414412,8.824212,-0.032073,-0.361850,0.133031,-3.282314,7.401748,2.888382,0.565048,13.572444,...,0.200000,0.100000,0.15,0.024305,0.246354,-0.325705,0.499206,0.533982,0.056462,1
3,0.504091,3.560697,0.052769,-0.219082,0.431470,-0.521901,2.131797,2.463859,0.761546,5.129187,...,0.047619,0.142857,0.00,-0.047120,-0.475068,0.567232,0.413155,-0.202047,-0.320971,1
4,0.670676,5.234183,0.129892,-0.309682,0.497022,-2.117456,2.900559,-1.493805,1.001392,8.021853,...,0.150000,0.050000,0.05,0.512207,0.258805,0.906142,0.860394,-0.914517,-0.957598,1
5,0.544915,5.744077,-0.019725,-0.160519,0.459703,-2.196203,5.137193,0.574986,0.692363,7.908382,...,0.050000,0.200000,0.05,-0.048445,0.519528,-0.267752,0.795589,-0.219352,-0.568584,1
6,0.437233,2.228335,0.044248,-0.173272,0.390459,-1.249536,0.604875,1.462136,0.617632,3.440446,...,0.050000,0.050000,0.15,0.831379,0.260701,0.264482,-0.254355,0.426677,-0.494492,1
7,0.512391,5.688852,0.167109,-0.289240,0.332999,-3.661801,1.755538,-0.809299,0.801389,8.823408,...,0.000000,0.100000,0.15,0.658285,0.785252,0.912541,-0.226515,0.146568,-0.920954,1
8,0.462571,3.478067,0.054085,0.046762,0.436618,-1.741785,1.647815,-1.134005,0.602086,4.996654,...,0.000000,0.100000,0.00,0.666688,0.770370,0.705055,-0.105500,0.246181,-0.950083,1
9,0.250030,1.937196,0.103667,-0.078559,0.186564,-0.108790,-0.088635,1.204102,0.368791,2.824890,...,0.150000,0.100000,0.15,0.007612,0.446691,-0.234656,0.336601,0.552242,-0.158663,1
